Load in required libraries, below.

In [1]:
# data 
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split


## SageMaker Resources

The below cell stores the SageMaker session and role (for creating estimators and models), and creates a default S3 bucket. After creating this bucket, locally stored data can be uploaded to S3.

In [2]:
# sagemaker
import boto3
import sagemaker
from sagemaker import get_execution_role

In [3]:
# SageMaker session and role
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

# default S3 bucket
bucket = sagemaker_session.default_bucket()
prefix='cnn-wendy-data-2'
prefix_output='cnn-wendy-model-2'

Here we retrieve the dataset of images and we upload it to S3


In [11]:
#!wget -nc https://da-youtube-ml.s3.eu-central-1.amazonaws.com/wendy-cnn/frames/wendy_cnn_frames_data_2.zip
#!unzip -qq -n wendy_cnn_frames_data_2.zip -d wendy_cnn_frames_data_2 

In [15]:

# upload to S3. Skip if already uploaded. This can take a while.
#print('Uploading data to {}'.format(prefix))
#input_data = sagemaker_session.upload_data(path='wendy_cnn_frames_data_3', bucket=bucket, key_prefix=prefix)
#print('Data uploaded to {}'.format(input_data))

In [16]:
# location to input data can be written down here, if known
input_data='s3://sagemaker-eu-central-1-283211002347/cnn-wendy-data-2'

After uploading images to S3, we can define and train the estimator


In [17]:
# import a PyTorch wrapper
from sagemaker.pytorch import PyTorch

# specify an output path

output_path = 's3://{}/{}'.format(bucket, prefix_output)
print('Output path for models is {}'.format(output_path))

# instantiate a pytorch estimator
estimator = PyTorch(entry_point='train.py',
                    source_dir='letsplay_classifier',
                    role=role,
                    framework_version='1.6',
                    train_instance_count=1,
                    train_instance_type='ml.p2.8xlarge',
                    train_volume_size = 10,
                    output_path=output_path,
                    sagemaker_session=sagemaker_session,
                    hyperparameters={
                        'img-width': 160,
                        'img-height': 90,
                        'batch-size': 32,
                        'layer-cfg': 'B',
                        'epochs': 6
                    })

Output path for models is s3://sagemaker-eu-central-1-283211002347/cnn-wendy-model-3


## Train the Estimator

After instantiating the estimator, we train it with a call to `.fit()`. 

In [ ]:
%%time 
# train the estimator on S3 training data
estimator.fit({'train': input_data})

'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


2020-10-25 04:52:37 Starting - Starting the training job...
2020-10-25 04:52:39 Starting - Launching requested ML instances.........
2020-10-25 04:54:32 Starting - Preparing the instances for training.........
2020-10-25 04:55:53 Downloading - Downloading input data...........................
2020-10-25 05:00:31 Training - Training image download completed. Training in progress.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2020-10-25 05:00:32,229 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2020-10-25 05:00:32,305 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2020-10-25 05:00:32,947 sagemaker_pytorch_container.training INFO     Invoking user training script.
2020-10-25 05:00:33,444 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
/opt/conda/bin/python -m pip install -r requirements.txt
2020-10-25 05:

In [ ]:
print(estimator.model_data)
model_data = estimator.model_data
model_data = 's3://sagemaker-eu-central-1-283211002347.s3.eu-central-1.amazonaws.com/cnn-wendy-model-2/pytorch-training-2020-10-25-04-52-37-645/output/model.tar.gz'

We set up a model that can predict the class of an image

### Deploy the trained model

We deploy our model to create a predictor. We'll use this to make predictions on our data and evaluate the model.

In [ ]:
# importing PyTorchModel
from sagemaker.pytorch import PyTorchModel

# Create a model from the trained estimator data
# And point to the prediction script
model = PyTorchModel(model_data=model_data,
                     role = role,
                     framework_version='1.6',
                     entry_point='predict.py',
                     source_dir='letsplay_classifier')

In [ ]:
%%time
# deploy and create a predictor
              
predictor = model.deploy(initial_instance_count=1, instance_type='ml.p2.xlarge')


In [ ]:
# the endpoint where the predictor is located
endpoint_name = predictor.endpoint
print(endpoint_name)

Now that the model is deployed, we check how the predictor performs on our full dataset,
ensuring that the predictions make sense. We produce a classification report.


In [4]:

endpoint_name='pytorch-inference-2020-10-25-06-03-13-786'

In [4]:

from sklearn.metrics import classification_report
from letsplay_classifier.endpoint import evaluate
y_true, y_pred = evaluate(endpoint_name, 'wendy_cnn_frames_data_2', 0.15)


KeyboardInterrupt: 

In [2]:
report = classification_report(y_true=y_true, y_pred=y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.95      0.99      0.97       927
           1       0.96      0.97      0.96       183
           2       0.99      0.98      0.99      4808
           3       0.94      0.97      0.96        35
           4       0.97      0.97      0.97      1050

    accuracy                           0.98      7003
   macro avg       0.96      0.98      0.97      7003
weighted avg       0.98      0.98      0.98      7003



In [3]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_true, y_pred)

array([[ 916,    0,    7,    1,    3],
       [   2,  177,    0,    1,    3],
       [  45,    8, 4733,    0,   22],
       [   0,    0,    1,   34,    0],
       [   2,    0,   31,    0, 1017]])

In [4]:
!wget -nc https://da-youtube-ml.s3.eu-central-1.amazonaws.com/wendy-cnn/frames/wendy_cnn_frames_E67.zip
!unzip -qq -n wendy_cnn_frames_E67.zip -d wendy_cnn_frames_E67

--2020-10-25 15:59:24--  https://da-youtube-ml.s3.eu-central-1.amazonaws.com/wendy-cnn/frames/wendy_cnn_frames_E67.zip
Resolving da-youtube-ml.s3.eu-central-1.amazonaws.com (da-youtube-ml.s3.eu-central-1.amazonaws.com)... 52.219.75.88
Connecting to da-youtube-ml.s3.eu-central-1.amazonaws.com (da-youtube-ml.s3.eu-central-1.amazonaws.com)|52.219.75.88|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 76964860 (73M) [application/zip]
Saving to: ‘wendy_cnn_frames_E67.zip’

wendy_cnn_frames_E6 100%[===================>]  73.40M  97.8MB/s    in 0.8s    

2020-10-25 15:59:25 (97.8 MB/s) - ‘wendy_cnn_frames_E67.zip’ saved [76964860/76964860]



In [ ]:
from letsplay_classifier.interval.predict_intervals_endpoint import evaluate
evaluate(endpoint_name, 'wendy_cnn_frames_E67/E67', class_names= ['Battle', 'Hideout', 'Other', 'Siege', 'Tournament'])

## Delete the Endpoint

Finally, I've added a convenience function to delete prediction endpoints after we're done with them. 

In [42]:
# Accepts a predictor endpoint as input
# And deletes the endpoint by name
def delete_endpoint(predictor):
        try:
            boto3.client('sagemaker').delete_endpoint(EndpointName=endpoint_name)
            print('Deleted {}'.format(predictor.endpoint))
        except:
            print('Already deleted: {}'.format(predictor.endpoint))

In [43]:
# delete the predictor endpoint 
delete_endpoint(predictor)

Deleted pytorch-inference-2020-10-23-05-01-33-314
